## Github Coursera Capston
This notebook will be mainly used for the capstone project.

#### Define related pakages

In [9]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#### Assignment 3 - Segmenting and Clustering Neighborhoods in Toronto

**Download List_of_postal_codes_of_Canada and read into data frame, and then rename column Postcode to PostalCode**

In [3]:
from pandas.io.html import read_html
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(url,  attrs={"class":"wikitable"},header =0)
wiki_df = pd.DataFrame(wikitable[0])
wiki_df= wiki_df.rename(columns={'Postcode':'PostalCode'})
wiki_df.head(11)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


 **Ignore cells with a borough that is Not assigned.**  
 **Sort by 'PostalCode','Borough'**

In [7]:
wiki_df_filtered_na = wiki_df[wiki_df['Borough']!="Not assigned"]
wiki_df_filtered_na=wiki_df_filtered_na.sort_values(by=['PostalCode','Borough'])
wiki_df_filtered_na.head()

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
26,M1C,Scarborough,Highland Creek
27,M1C,Scarborough,Rouge Hill
28,M1C,Scarborough,Port Union


 **Combine rows group by postalcode and neighborhoods separated  with a comma**

In [8]:
wiki_group = wiki_df_filtered_na.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
print(wiki_group.shape)

(103, 3)


In [106]:
wiki_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [109]:
## Manipulate N/A value of Neightborhood by using Borough
wiki_group.loc[wiki_group["Neighborhood"]=="Not assigned", "Neighborhood"] = wiki_group.loc[wiki_group["Neighborhood"]=="Not assigned", "Borough"]
wiki_group.shape

(103, 3)

## Extract Coordinators from "Geospatial_Coordinates.csv"

In [110]:
latlong = pd.read_csv("Geospatial_Coordinates.csv")
latlong=latlong.rename(columns={'Postal Code':'PostalCode'})
latlong.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge coordinators information into dataframe

In [113]:
wiki_group_geo=pd.merge(wiki_group, latlong, on='PostalCode')
wiki_group_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Get latitude Longitude of Toronto, Ontario'

In [115]:
geolocator = Nominatim()
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


## Maps visualizations represent location 

In [116]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
 
for lat, long, post, borough, neigh in zip(wiki_group_geo['Latitude'], wiki_group_geo['Longitude'], wiki_group_geo['PostalCode'], wiki_group_geo['Borough'], wiki_group_geo['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#00FF00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

![Toronto_MAp](map.png?raw=true)

In [122]:
CLIENT_ID = ' ' 
CLIENT_SECRET = ' '
VERSION = ' '

### Use Foursquare API to explore venue nearby 

In [140]:
import requests


toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_central_df = wiki_group_geo[wiki_group_geo['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print(toronto_central_df.shape)
toronto_central_df.head()

radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['PostalCode'], toronto_central_df['Borough'], toronto_central_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

(39, 5)


In [141]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1720, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [142]:
toronto_central_dummy = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_central_dummy['PostalCode'] = venues_df['PostalCode'] 
toronto_central_dummy['Borough'] = venues_df['Borough'] 
toronto_central_dummy['Neighborhoods'] = venues_df['Neighborhood'] 

columns = list(toronto_central_dummy.columns[-3:]) + list(toronto_central_dummy.columns[:-3])
toronto_central_dummy = toronto_central_dummy[columns]

print(toronto_central_dummy.shape)
toronto_central_dummy.head()

(1720, 238)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West,Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
toronto_central_venues = toronto_central_dummy.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(toronto_central_venues.shape)
toronto_central_venues.head()

(39, 238)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.023256,...,0.023256,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.023256
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,...,0.000000,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.023810
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000


## Add nearby venues in dataframe

In [145]:
num_venues = 10

indicators = ['st', 'nd', 'rd']

area = ['PostalCode', 'Borough', 'Neighborhoods']
freq = []
for i in np.arange(num_venues):
    try:
        freq.append('{}{} Venue'.format(i+1, indicators[ind]))
    except:
        freq.append('{}th Venue'.format(i+1))
columns = area+freq
# create a new dataframe
neighborhoods_venues = pd.DataFrame(columns=columns)
neighborhoods_venues['PostalCode'] = toronto_central_venues['PostalCode']
neighborhoods_venues['Borough'] = toronto_central_venues['Borough']
neighborhoods_venues['Neighborhoods'] = toronto_central_venues['Neighborhoods']

for i in np.arange(toronto_central_venues.shape[0]):
    row_categories = toronto_central_venues.iloc[i, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues.iloc[i, 3:] = row_categories_sorted.index.values[0:num_venues]

neighborhoods_venues.sort_values(freq, inplace=True)
neighborhoods_venues.head()

,PostalCode,Borough,Neighborhoods,1th Venue,2th Venue,3th Venue,4th Venue,5th Venue,6th Venue,7th Venue,8th Venue,9th Venue,10th Venue
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique
31,M6H,West Toronto,"Dovercourt Village,Dufferin",Bakery,Pharmacy,Supermarket,Pizza Place,Music Venue,Portuguese Restaurant,Recording Studio,Middle Eastern Restaurant,Café,Brewery
32,M6J,West Toronto,"Little Portugal,Trinity",Bar,Asian Restaurant,Restaurant,Pizza Place,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Men's Store,Café,Juice Bar
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Brewery,Yoga Studio,Sandwich Place,Fish Market
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gym,Bakery,Performing Arts Venue,Pet Store,Grocery Store,Climbing Gym,Restaurant


## Create venues cluster using venues with K=3

In [149]:
k = 3

toronto_central_venues_clustering = toronto_central_venues.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_central_venues_clustering)

toronto_central_clustered_df = toronto_central_df
toronto_central_clustered_df['Cluster'] = kmeans.labels_

toronto_central_clustered_df = toronto_central_clustered_df.join(neighborhoods_venues.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_central_clustered_df.sort_values(['Cluster'] + freq, inplace=True)
toronto_central_clustered_df["Cluster"].value_counts()

0    36
2     2
1     1
Name: Cluster, dtype: int64

## Display cluster on Map

In [152]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_central_clustered_df['Latitude'], toronto_central_clustered_df['Longitude'], toronto_central_clustered_df['PostalCode'], toronto_central_clustered_df['Borough'], toronto_central_clustered_df['Neighborhood'], toronto_central_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

![Toronto_Cluster](cluster.png?raw=true)